In [48]:
from typing import TYPE_CHECKING, Generic, Hashable, Iterable, Optional, Union

import numpy as np
import xarray as xr
from xarray.core.weighted import Weighted

import xcollection
from xcollection import Collection

In [160]:
ds = xr.tutorial.open_dataset('rasm')
dsa = xr.tutorial.open_dataset('air_temperature')
ds_dict = {'foo': ds, 'bar': dsa}
collection = xcollection.Collection(ds_dict)

In [166]:
len(ds.Tair)

36

In [170]:
weights = np.cos(ds.Tair)
weights.name = "weights"
weights

<xarray.DataArray 'weights' (time: 36, y: 205, x: 275)>
array([[[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [        nan,         nan,         nan, ..., -0.32344126,
         -0.32188526, -0.59058038],
        [        nan,         nan,         nan, ..., -0.54372379,
         -0.0989191 , -0.37323875],
        [        nan,         nan,         nan, ..., -0.31447777,
          0.1383537 , -0.02710846]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
...
        [        nan,         nan,         nan, ..., -0.91528289,
         -0.90258936, -0.98351232],
        [        nan,         nan,         nan, ..., -0.92966533,
         -0.57082628, -0.82507456],
        [        nan,         nan,         nan, ..., -0.69241693,
         -0.30063353, -0.50263337]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [        nan,         nan,         nan, ..., -0.77444636,
         -0.82398574, -0.71471853],
        [        nan,         nan,         nan, ..., -0.72359608,
         -0.98900863, -0.91596978],
        [        nan,         nan,         nan, ..., -0.92510589,
         -0.99609716, -0.99776946]]])
Coordinates:
  * time     (time) object 1980-09-16 12:00:00 ... 1983-08-17 00:00:00
    xc       (y, x) float64 189.2 189.4 189.6 189.7 ... 17.65 17.4 17.15 16.91
    yc       (y, x) float64 16.53 16.78 17.02 17.27 ... 28.26 28.01 27.76 27.51
Dimensions without coordinates: y, x
Attributes:
    units:           C
    long_name:       Surface air temperature
    type_preferred:  double
    time_rep:        instantaneous

In [6]:
weighted = {key: ds.weighted(weights.fillna(0)) for key, ds in c.items()}
print(weighted)

{'foo': DatasetWeighted with weights along dimensions: time, y, x, 'bar': DatasetWeighted with weights along dimensions: time, y, x}


In [55]:
class CollectionWeighted(Weighted["Collection"]):
    def _check_dim(self, dim: Optional[Union[Hashable, Iterable[Hashable]]]):
        """raise an error if any dimension is missing"""

        for key, dataset in self.obj.items():
            if isinstance(dim, str) or not isinstance(dim, Iterable):
                dims = [dim] if dim else []
            else:
                dims = list(dim)
            missing_dims = set(dims) - set(dataset.dims) - set(self.weights.dims)
            if missing_dims:
                raise ValueError(
                    f"{dataset.__class__.__name__} does not contain the dimensions: {missing_dims}"
                )

    def _implementation(self, func, dim, **kwargs) -> "Collection":

        self._check_dim(dim)

        dataset_dict = {}
        for key, dataset in self.obj.items():

            dataset = dataset.map(func, dim=dim, **kwargs)
            dataset_dict[key] = dataset
        return Collection(dataset_dict)

In [56]:
cw = CollectionWeighted(c, weights.fillna(0))

In [155]:
cwm = cw.mean(dim="time")

In [156]:
ds = xr.tutorial.open_dataset('rasm')
dsa = xr.tutorial.open_dataset('air_temperature')
ds_dict = {'foo': ds, 'bar': dsa}
collection = xcollection.Collection(ds_dict)

weights = np.cos(np.deg2rad(ds.Tair))
weights.name = "weights"

collection_weighted = CollectionWeighted(collection, weights.fillna(0))
collection_dict = {
    "mean": collection_weighted.mean(dim="time"),
    "sum": collection_weighted.sum(dim="time"),
    "sum_of_weights": collection_weighted.sum_of_weights(dim="time"),
}

dict_weighted = {key: ds_dict[key].weighted(weights.fillna(0)) for key in ds_dict}
dict_dict = {
    "mean": {key: dict_weighted[key].mean(dim="time") for key in dict_weighted},
    "sum": {key: dict_weighted[key].sum(dim="time") for key in dict_weighted},
    "sum_of_weights": {key: dict_weighted[key].sum_of_weights(dim="time") for key in dict_weighted},
}

for k in collection_dict:
    for j, ds in collection_dict[k].items():
        assert type(ds) == type(dict_dict[k][j])
        assert ds == dict_dict[k][j]
        print(type(ds))

<class 'xarray.core.dataset.Dataset'>
<class 'xarray.core.dataset.Dataset'>
<class 'xarray.core.dataset.Dataset'>
<class 'xarray.core.dataset.Dataset'>
<class 'xarray.core.dataset.Dataset'>
<class 'xarray.core.dataset.Dataset'>
